In [1]:
import pandas as pd

# Membaca file CSV
data = pd.read_csv('tweets.csv')
teks = data['tweet']  # Ganti 'kolom_teks' dengan nama kolom yang sesuai

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

def preprocessing(teks):
    teks = re.sub(r'\W', ' ', teks)  # Menghapus karakter non-kata
    teks = teks.lower()  # Mengubah ke huruf kecil
    teks = word_tokenize(teks)  # Tokenisasi
    teks = [word for word in teks if word not in stop_words]  # Menghapus stopwords
    return ' '.join(teks)

cleaned_texts = teks.apply(preprocessing)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elsam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from gensim import corpora
from gensim.models import LdaModel

# Membuat dictionary dan corpus
dictionary = corpora.Dictionary(cleaned_texts.str.split())
corpus = [dictionary.doc2bow(text.split()) for text in cleaned_texts]

# Menerapkan LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [4]:
for idx, topic in lda_model.print_topics(-1):
    print(f'Topik {idx}: {topic}')

Topik 0: 0.062*"ui" + 0.062*"green" + 0.062*"administrasi" + 0.062*"fia" + 0.062*"ilmu" + 0.062*"berkonsep" + 0.062*"fakultas" + 0.062*"building" + 0.062*"meresmikan" + 0.061*"gedung"
Topik 1: 0.025*"indonesia" + 0.025*"universitas" + 0.025*"rektor" + 0.025*"soemantri" + 0.025*"brodjonegoro" + 0.025*"teknologi" + 0.025*"pendidikan" + 0.017*"menteri" + 0.017*"mendiktisaintek" + 0.017*"ayahnya"
Topik 2: 0.061*"desa" + 0.061*"kampus" + 0.059*"riza" + 0.059*"wamendes" + 0.059*"ahmad" + 0.059*"keterlibatan" + 0.059*"patria" + 0.059*"percepatan" + 0.059*"pdt" + 0.059*"pembangunan"
Topik 3: 0.031*"univ_indonesia" + 0.031*"pati" + 0.017*"yg" + 0.017*"ui" + 0.017*"ca" + 0.017*"kasih" + 0.017*"jd" + 0.017*"ccccaacaca" + 0.017*"paham" + 0.017*"lu"
Topik 4: 0.057*"co" + 0.057*"https" + 0.057*"t" + 0.049*"rektor" + 0.046*"indonesia" + 0.034*"universitas" + 0.023*"selamat" + 0.016*"2025" + 0.016*"hermansyah" + 0.016*"heri"


In [5]:
topic_distribution = [lda_model.get_document_topics(bow) for bow in corpus]
# Analisis lebih lanjut untuk menemukan topik yang paling banyak dibahas

In [8]:
from collections import Counter

# Menghitung frekuensi kemunculan setiap topik
topic_counts = Counter()
for doc_topics in topic_distribution:
    for topic, _ in doc_topics:
        topic_counts[topic] += 1

# Menampilkan 5 topik teratas
top_5_topics = topic_counts.most_common(5)
print("5 Topik Paling Banyak Dibahas:")
for topic, count in top_5_topics:
    print(f'Topik {topic}: {count} dokumen')

# Menampilkan kata kunci untuk setiap topik
print("\nKata Kunci untuk Setiap Topik:")
for idx in range(5):  # Hanya untuk 5 topik teratas
    print(f'Topik {idx}: {lda_model.print_topics(num_words=3)[idx][1]}')

# Menampilkan dokumen yang paling relevan untuk setiap topik teratas
print("\nDokumen Relevan untuk Topik Teratas:")
for topic, _ in top_5_topics:
    print(f'\nDokumen untuk Topik {topic}:')
    relevant_docs = [teks[i] for i, doc_topics in enumerate(topic_distribution) if any(t == topic for t, _ in doc_topics)]
    for doc in relevant_docs[:2]:  # Menampilkan 3 dokumen relevan
        print(f'- {doc}')

5 Topik Paling Banyak Dibahas:
Topik 4: 76 dokumen
Topik 2: 75 dokumen
Topik 1: 74 dokumen
Topik 0: 73 dokumen
Topik 3: 73 dokumen

Kata Kunci untuk Setiap Topik:
Topik 0: 0.062*"ui" + 0.062*"green" + 0.062*"administrasi"
Topik 1: 0.025*"indonesia" + 0.025*"universitas" + 0.025*"rektor"
Topik 2: 0.061*"desa" + 0.061*"kampus" + 0.059*"riza"
Topik 3: 0.031*"univ_indonesia" + 0.031*"pati" + 0.017*"yg"
Topik 4: 0.057*"co" + 0.057*"https" + 0.057*"t"

Dokumen Relevan untuk Topik Teratas:

Dokumen untuk Topik 4:
- RT @datatempo: Ternyata ayahnya Menteri Pendidikan Tinggi, Sains, dan Teknologi (Mendiktisaintek) Satryo Soemantri Brodjonegoro pernah menj…
- Ternyata ayahnya Menteri Pendidikan Tinggi, Sains, dan Teknologi (Mendiktisaintek) Satryo Soemantri Brodjonegoro pernah menjadi Rektor Universitas Indonesia, yaitu SOEMANTRI BRODJONEGORO  - DATATEMPO https://t.co/Q2gQ3W8QyB

Dokumen untuk Topik 2:
- RT @datatempo: Ternyata ayahnya Menteri Pendidikan Tinggi, Sains, dan Teknologi (Mendiktisain

In [9]:
import pyLDAvis.gensim_models
import pyLDAvis

# Visualisasi model LDA
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

In [10]:
for idx, topic in lda_model.print_topics(num_words=10):
    print(f'Topik {idx}: {topic}')

Topik 0: 0.062*"ui" + 0.062*"green" + 0.062*"administrasi" + 0.062*"fia" + 0.062*"ilmu" + 0.062*"berkonsep" + 0.062*"fakultas" + 0.062*"building" + 0.062*"meresmikan" + 0.061*"gedung"
Topik 1: 0.025*"indonesia" + 0.025*"universitas" + 0.025*"rektor" + 0.025*"soemantri" + 0.025*"brodjonegoro" + 0.025*"teknologi" + 0.025*"pendidikan" + 0.017*"menteri" + 0.017*"mendiktisaintek" + 0.017*"ayahnya"
Topik 2: 0.061*"desa" + 0.061*"kampus" + 0.059*"riza" + 0.059*"wamendes" + 0.059*"ahmad" + 0.059*"keterlibatan" + 0.059*"patria" + 0.059*"percepatan" + 0.059*"pdt" + 0.059*"pembangunan"
Topik 3: 0.031*"univ_indonesia" + 0.031*"pati" + 0.017*"yg" + 0.017*"ui" + 0.017*"ca" + 0.017*"kasih" + 0.017*"jd" + 0.017*"ccccaacaca" + 0.017*"paham" + 0.017*"lu"
Topik 4: 0.057*"co" + 0.057*"https" + 0.057*"t" + 0.049*"rektor" + 0.046*"indonesia" + 0.034*"universitas" + 0.023*"selamat" + 0.016*"2025" + 0.016*"hermansyah" + 0.016*"heri"
